In [1]:
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from datasets import load_dataset

from dotenv import load_dotenv

import json
import pandas as pd
import os

from azure.identity import (
    DefaultAzureCredential,
)

from azure.identity import AzureAuthorityHosts
from azure.keyvault.secrets import SecretClient

### Week 1 Homework
In your Azure account create 
- An Azure OpenAI instance 
- Deploy a model and configure capacity
- A Key Vault
- Setup the .env file
- Test connectivity between your laptop and the AOAI instance 
- Run a prompt against the AOAI instance
- Create a Synapse instance

### Create your Azure OpenAI Resource and Key Vault

Navigate to the [Azure Portal](https://portal.azure.com/#home) or [US Gov Azure Portal](https://portal.azure.us/#home) and login using your account. Next you're going to create an Azure OpenAI resource (use the AZ region), create a new resource group and use any unique name for the resource's name.  
Once the resource is created, you need to open [Azure AI Foundry](https://ai.azure.com/) or [Azure OpenAI Studio](https://ai.azure.us/) to deploy the model. Navigate to deployments, press deploy model and select gpt-4o-mini.  Make sure to increase your rate limit, or tokens per minute (around 700k should be sufficient)

Once that is created, copy the key (Found under Resource Management > Keys and Endpoints) and create a new key vault. Assign the same resource group as your Azure OpenAI resource and again pick a unique name for the key vault name.  

Once the key vault is created, make a new secret with the API key.  

### Connect to Azure OpenAI
First, run `az login` in the terminal and login to your FedAIRS account.  
<br>
If you are using a Gov account:<br>
az cloud set --name AzureUSGovernment <br>
az login <br>
az account set --subscription="your subscription"<br>

If you are using a commercial account:<br>
az login <br>
az account set --subscription="your subscription"<br>
<br>
Two things are needed to connect to your Azure OpenAI resource
- Your API key
- Your Endpoint  
  
For the API Key, we are going to connect to the key vault we just made to insert the key. For this, you'll need to change the URL below to match your key vault's URL.  
Next, we will insert the endpoint URL from our Azure OpenAI resource.

Our `azure_client` is where we are calling the LLM and connecting to the model we deployed. Other parameters can be passed in, like timeout or max_retires.

Note: In the block below we inserted `credential = DefaultAzureCredential(authority=AzureAuthorityHosts.AZURE_GOVERNMENT)`. This is because when we use our FedAIRS account we are connected to the US Government cloud, rather than the regular commercial cloud.

### Create a .env File
To secure your secrets with checking them into a repository, create a .env to contain secrets that are specific to your project.  The file will look like this:

KEY_VAULT_URL = "The key vault URL"<br>
SECRET_NAME = "name of the secret for the AOAI API key"<br>
AZURE_OPENAI_ENDPOINT = "your AOAI endpoint"<br>
API_VERSION = "The API version.  The latest version is: 2024-10-21"<br>
DEPLOYMENT = "your deployment name"<br>

Be sure to add .env to your gitignore file

### Create and configure deployment

Deploy a gpt-4o-mini model and set the capacity


In [5]:
load_dotenv()

credential = DefaultAzureCredential(authority=AzureAuthorityHosts.AZURE_GOVERNMENT)

secret_client = SecretClient(vault_url=os.getenv('KEY_VAULT_URL'), credential=credential)
deployment = os.getenv('DEPLOYMENT')
endpoint_url = os.getenv('AZURE_OPENAI_ENDPOINT')
api_version = os.getenv('API_VERSION')
api_key = secret_client.get_secret(os.getenv('SECRET_NAME')).value


azure_client = AzureChatOpenAI(
                api_key=api_key
                ,api_version=api_version
                ,azure_endpoint=endpoint_url
                ,deployment_name=deployment
                ,temperature=0
                ,max_tokens=4000
                ,model_kwargs={"response_format": {"type": "json_object"}}
)

In [6]:
doc_prompt = PromptTemplate.from_template(
    template = '''

Write a poem about {input}.  Return your results in json

'''
)

In [ ]:
doc_chain = (
    doc_prompt
    | azure_client
    | StrOutputParser()
)

print(doc_chain.invoke({"input": "unicorns"}))

### Estimate Costs

Calculate the estimated costs in the text field of the Symptom2Disease dataset
